In [9]:
import pandas as pd
import numpy as np
import requests
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from elasticsearch import Elasticsearch

model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

/Users/shun_lung_chang/python_projects/llm_zoomcamp/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Question 1

In [2]:
user_question = "I just discovered the course. Can I still join it?"

v = embedding_model.encode(user_question)
v[0]

np.float32(0.07822262)

# Question 2

In [26]:
base_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main"
relative_url = "03-vector-search/eval/documents-with-ids.json"
docs_url = f"{base_url}/{relative_url}?raw=1"
docs_response = requests.get(docs_url)
documents = docs_response.json()

documents = [
    document
    for document in documents
    if document["course"] == "machine-learning-zoomcamp"
]

In [37]:
embeddings = []

for document in tqdm(documents):
    qa_text = f'{document["question"]} {document["text"]}'
    qa_vector = embedding_model.encode(qa_text)
    embeddings.append(qa_vector)
    document["qa_vector"] = qa_vector


X = np.array(embeddings)
X.shape

100%|██████████| 375/375 [00:05<00:00, 68.42it/s]


(375, 768)

# Question 3

In [6]:
max(X @ v)

np.float32(0.6506574)

# Question 4

In [7]:
class VectorSearchEngine:
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]


def hit_rate(relevance_total: list[list[bool]]) -> float:
    count = 0

    for line in relevance_total:
        if True in line:
            count += 1

    return count / len(relevance_total)


base_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main"
relative_url = "03-vector-search/eval/ground-truth-data.csv"
ground_truth_url = f"{base_url}/{relative_url}?raw=1"

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == "machine-learning-zoomcamp"]
ground_truth = df_ground_truth.to_dict(orient="records")

In [63]:
search_engine = VectorSearchEngine(documents=documents, embeddings=X)

In [64]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q["document"]
    question_vector = embedding_model.encode(q["question"])
    results = search_engine.search(question_vector, num_results=5)
    relevance = [d["id"] == doc_id for d in results]
    relevance_total.append(relevance)

hit_rate(relevance_total)

100%|██████████| 1830/1830 [00:16<00:00, 109.72it/s]


0.9398907103825137

# Question 5

In [11]:
es_client = Elasticsearch("http://localhost:9200")
es_client.info()

ObjectApiResponse({'name': 'dda37d88da65', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'bRp33vekT_yG9gFTuwVO0g', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [30]:
embedding_dims = X.shape[1]
index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "qa_vector": {
                "type": "dense_vector",
                "dims": embedding_dims,
                "index": True,
                "similarity": "cosine",
            },
        }
    },
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [41]:
for document in tqdm(documents):
    doc = {"id": document["id"], "qa_vector": document["qa_vector"]}
    es_client.index(index=index_name, document=doc)

100%|██████████| 375/375 [00:01<00:00, 204.41it/s]


In [52]:
def elastic_search_knn(query_vector: np.array):
    knn = {
        "field": "qa_vector",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 10000,
    }

    search_query = {"knn": knn, "_source": ["id"]}

    es_results = es_client.search(index=index_name, body=search_query)

    return [hit["_source"] for hit in es_results["hits"]["hits"]]

In [60]:
elastic_search_knn(v)[0]

{'id': 'ee58a693'}

In [61]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q["document"]
    question_vector = embedding_model.encode(q["question"])
    results = elastic_search_knn(question_vector)
    relevance = [d["id"] == doc_id for d in results]
    relevance_total.append(relevance)

100%|██████████| 1830/1830 [00:41<00:00, 44.07it/s]


In [62]:
hit_rate(relevance_total)

0.9398907103825137